In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
config = Config(debug=True)
torch.cuda.set_device(config.gpu_start)

In [ ]:
postpond = 50
fns_feathers = sorted(config.fn_cycles.glob('cycle*'))[:config.num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
# idx = [beginning_index(o.iloc[:500]) + postpond for o in tqdm_notebook(cycles)]

In [ ]:
deps = fmtr.get_y(dep_var)

In [ ]:
import seaborn as sns

In [ ]:
deps.head()

In [ ]:
y1 = deps.iloc[:,0]
ax = sns.distplot(y1)
set_font(ax.xaxis.label)

In [ ]:
y1 = deps.iloc[:,1]
ax = sns.distplot(y1)
set_font(ax.xaxis.label)

In [ ]:
# cont_names = [o for o in cycles[0].columns[2:2+n_cont] if o not in dep_var]
cont_names = cont_names if selected_columns else [o for o in cycles[0].columns[2:] if o not in dep_var]
df_conts = tile_with_noise(cycles, idx, mulr, cont_names)

In [ ]:
deps = fmtr.get_y(dep_var)
deps = pd.concat([deps]*mulr).reset_index(drop=True)
cyc_cont = flatten_and_cat(df_conts, deps)

In [ ]:
cyc_cont.to_feather('tmp/cyc_cont_all_6_debug')

In [ ]:
#init
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all_6_debug')

In [ ]:
#init
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all_allc')

In [ ]:
valid_idx_tile = (valid_idx[:,None] + np.arange(mulr) * num_cycles).flatten() # take from all tiles
# procs = []
procs = [Normalize]
databunch = MultiDeptTabularDataBunch.from_df('tmp', cyc_cont, dep_var, valid_idx=valid_idx_tile, bs=bs, procs=procs)

rnndb = RNNDataBunch.create(databunch.train_ds, databunch.valid_ds, bs=bs)

In [ ]:
n_hidden, n_layers = n_cont * 3, 3
# hidden_p, input_p, embed_p, weight_p = 0,0,0,0
hidden_p, input_p, weight_p = 0.3, 0.6, 0.5

rnn_enc = ContModel1(1, n_cont, n_hidden, n_layers, 
                     hidden_p=hidden_p, input_p=input_p, embed_p=0, weight_p=weight_p)

layers = [3*n_cont, 5, 2]
# drops = [0, 0, 0, 0]
drops = [0.1] * 4
# model = MultiInputSequentialRNN(rnn_enc, DummyModel(layers, drops)).cuda()
# model = DummyContModel1(1, n_cont, n_hidden, n_layers)
model = MultiInputSequentialRNN(rnn_enc, PoolingLinearClassifier(layers, drops)).cuda()
model.reset()
    
weight = cyc_cont[dep_var].max().values
weight = torch.tensor(weight[::-1] / weight.max(), dtype=torch.float32).cuda()

# learner = Learner(rnndb, model, loss_func=weighted_rnn_mse(weight), metrics=rnn_metrics, opt_func=optim.SGD)
learner = Learner(rnndb, model, loss_func=MSELossFlat(), metrics=rnn_metrics, opt_func=optim.SGD)
# learner = Learner(rnndb, model, loss_func=weighted_our_loss(weight), metrics=rnn_metrics, opt_func=optim.SGD)

from fastai.callbacks import *
# learner.callback_fns += [ShowGraph, partial(SaveModelCallback, name='rnn0')]
learner.callback_fns += [ShowGraph,]
learner.callbacks += [TerminateOnNaNCallback()]
# alpha, beta = 2., 1.
alpha, beta = 1., 1.
# alpha, beta = 0., 0.
learner.callbacks.append(RNNTrainer(learner, sl, alpha=alpha, beta=beta))

clip = 0.5
learner.callback_fns.append(partial(GradientClipping, clip=clip))
split_func = lambda x: list(x)
# learner.split(split_func)
learner.split(rnn_classifier_split)

In [ ]:
learner.callback_fns[-1].keywords['clip'] = 1e1

In [ ]:
len(learner.layer_groups)

In [ ]:
learner.layer_groups

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=100)
learner.recorder.plot()

In [ ]:

learner.fit_one_cycle(100, lr)

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=100)
learner.recorder.plot()

In [ ]:
learner.fit(100, lr / 2)

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=100)
learner.recorder.plot()

In [ ]:
learner.callback_fns[-1].keywords['clip'] = .1
learner.callback_fns[-1].keywords

In [ ]:
learner.fit(20, 1e-1)

In [ ]:
learner.fit(30, 1e-2)

In [ ]:
it = iter(learner.data.valid_dl)

In [ ]:
# x, y = ni(learner.data.train_dl)
# x, y = ni(learner.data.valid_dl)
x, y = next(it)
y, p = y.cpu(), learner.model(*x)[0].cpu()
y_np, p_np = to_np(y), to_np(p)
# our_metrics(y, p), our_metrics_np(y_np, p_np)
lf = weighted_rnn_mse(weight.cpu())
our_metrics_np(p_np, y_np)

In [ ]:
np.argmin(np.abs(y_np-p_np) / (y_np + 1e-12), axis=0)

In [ ]:
y_np[237,0] = p_np[237,0]

In [ ]:
p_np[130]

In [ ]:
(np.abs(y_np-p_np) / (y_np+1e-12)).sum() / y_np.size

In [ ]:
x[1].shape

In [ ]:
p

In [ ]:
i = 0
list(zip(p_np[:,i], y_np[:,i]))

In [ ]:
p = learner.model(*x)[1]

In [ ]:
# l x b x t x h
l, b = 0, 0
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for t in range(30)]

In [ ]:
# l x b x t x h
l, b, t = 0, 0, 1
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for l in range(n_layers)]

In [ ]:
list(zip(y_np.tolist(), p_np.tolist()))